In [61]:
import os
import json
import math
import numpy as np 
import pandas as pd


## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline 
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgb
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
from skimage.util import random_noise

## PyTorch
## Backprop Learning
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
torch.cuda.empty_cache()
# Torchvision
import torchvision
from torchvision.datasets import ImageFolder
from torchvision import transforms

## Hebbian Learning + Backprop
!pip install nupic.torch-master.zip
from nupic.torch.modules import (
    KWinners2d, KWinners, SparseWeights, SparseWeights2d, Flatten,
    rezero_weights, update_boost_strength
)

Defaulting to user installation because normal site-packages is not writeable
Processing c:\mgr materialy + praca mgr\4 + 2 semestr\ssne\mp5\nupic.torch-master.zip
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build nupic.torch


ERROR: Could not build wheels for nupic.torch, which is required to install pyproject.toml-based projects

[notice] A new release of pip available: 22.2.1 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
C:\Users\piomi\AppData\Local\Temp\ipykernel_8708\2023843792.py:12: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg', 'pdf') # For export


Defaulting to user installation because normal site-packages is not writeable
Processing c:\mgr materialy + praca mgr\4 + 2 semestr\ssne\mp5\nupic.torch-master.zip
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build nupic.torch


ERROR: Could not build wheels for nupic.torch, which is required to install pyproject.toml-based projects

[notice] A new release of pip available: 22.2.1 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [62]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

Device: cuda:0


In [63]:
# Transformations applied on each image => only make them a tensor
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                ])

# Loading the training dataset. We need to split it into a training and validation part
dataset = ImageFolder("trafic_32", transform=transform)
#dataset_t = ImageFolder("trafic_32", transform=transform_t)
#dataset_enhanced = torch.utils.data.ConcatDataset([dataset, dataset_t])

In [64]:
batch_size = 128
train_loader = data.DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True, pin_memory=True, num_workers=2)

In [65]:
# Sparsity parameters
SPARSITY = 0.66
SPARSITY_CNN = SPARSITY

# K-Winners parameters
PERCENT_ON = 0.5
BOOST_STRENGTH = 1.8
relu_flag = False

# Models
latent_dim = 200
img_size = 32 * 32 * 3

# W-GAN PARAMETERS
WEIGHT_CLIP = 0.01
disc_iter = 5

# Layers
drop_out = 0.2

In [66]:
class Cond_Generator(nn.Module):
    def __init__(self, batch_size,latent_dim, hidden_dim, output_dim, num_of_classes,SPARSITY_CNN,BOOST_STRENGTH,PERCENT_ON,relu_flag):
        super(Cond_Generator, self).__init__()
        self.batch_size = batch_size
        self.latent_dim = latent_dim
        self.hidden_dim = hidden_dim
        self.output_dim = int(output_dim)
        self.num_of_classes = num_of_classes
        self.relu_flag= relu_flag
        self.SPARSITY_CNN = SPARSITY_CNN
        #
        # self.fc_1 = nn.Sequential(nn.Linear(self.latent_dim+self.num_of_classes, self.latent_dim+self.num_of_classes))
        #
        # # self.fc_2 = nn.Sequential(SparseWeights(nn.Linear(self.hidden_dim, self.hidden_dim), sparsity=SPARSITY),
        # # KWinners(n=self.hidden_dim, percent_on=PERCENT_ON, boost_strength=BOOST_STRENGTH,relu=self.relu_flag),)
        #
        # self.fc_2 = nn.Sequential(nn.Linear(self.latent_dim+self.num_of_classes, int(self.hidden_dim)))
        # self.LeakyReLU = nn.LeakyReLU(0.2)

        self.cntv_1 = nn.Sequential(
            nn.ConvTranspose2d(int(self.latent_dim), int(self.hidden_dim*4), kernel_size=(3, 3), stride=(1, 1), padding=(0, 0), bias=False),
            KWinners2d(channels=int(self.hidden_dim*4), percent_on=PERCENT_ON, boost_strength=BOOST_STRENGTH,relu=self.relu_flag),
            nn.Dropout(drop_out)
        )

        self.cntv_labels = nn.Sequential(
            nn.ConvTranspose2d(int(self.num_of_classes), int(self.hidden_dim*4), kernel_size=(3, 3), stride=(1, 1), padding=(0, 0), bias=False),
            KWinners2d(channels=int(self.hidden_dim*4), percent_on=PERCENT_ON, boost_strength=BOOST_STRENGTH,relu=self.relu_flag),
            nn.Dropout(drop_out)
        )

        self.cntv_2 = nn.Sequential(
            nn.ConvTranspose2d( int(self.hidden_dim*8), int(self.hidden_dim*4), kernel_size=(3,3), stride=(2,2), padding=(1,1), bias=False),
            nn.BatchNorm2d(int(self.hidden_dim*4)),
            #nn.InstanceNorm2d(int(self.hidden_dim*4)),
            KWinners2d(channels=int(self.hidden_dim*4), percent_on=PERCENT_ON, boost_strength=BOOST_STRENGTH,relu=self.relu_flag),
            nn.Dropout(drop_out)
        )

        self.cntv_3 = nn.Sequential(
            nn.ConvTranspose2d(int(self.hidden_dim*4), int(self.hidden_dim*2), kernel_size=(3,3), stride=(2,2), padding=(1,1), bias=False),
            nn.BatchNorm2d(int(self.hidden_dim*2)),
            #nn.InstanceNorm2d(int(self.hidden_dim*2)),
            KWinners2d(channels=int(self.hidden_dim*2), percent_on=PERCENT_ON, boost_strength=BOOST_STRENGTH,relu=self.relu_flag),
            nn.Dropout(drop_out),
        )

        self.cntv_4 = nn.Sequential(
            nn.ConvTranspose2d(int(self.hidden_dim*2), int(self.hidden_dim), kernel_size=(3,3), stride=(2,2), padding=(1,1), bias=False),
            nn.BatchNorm2d(int(self.hidden_dim)),
            #nn.InstanceNorm2d(int(self.hidden_dim)),
            KWinners2d(channels=int(self.hidden_dim), percent_on=PERCENT_ON, boost_strength=BOOST_STRENGTH,relu=self.relu_flag),
            nn.Dropout(drop_out)
        )

        self.cntv_5 =  nn.Sequential(
            nn.ConvTranspose2d(int(self.hidden_dim), int(self.hidden_dim/2), kernel_size=(3,3), stride=(2,2), padding=(1,1), bias=False),
            nn.BatchNorm2d(int(self.hidden_dim/2)),
            #nn.InstanceNorm2d(int(self.hidden_dim/2)),
            KWinners2d(channels=int(self.hidden_dim/2), percent_on=PERCENT_ON, boost_strength=BOOST_STRENGTH,relu=self.relu_flag),
            nn.Dropout(drop_out)
        )

        self.cntv_6 = nn.Sequential(
            nn.ConvTranspose2d(int(self.hidden_dim/2), int(self.output_dim/32/32), kernel_size=(4,4), stride=(1,1), padding=(2,2), bias=False),
            nn.BatchNorm2d(int(self.output_dim/32/32)),#https://arxiv.org/pdf/1805.07389.pdf -> in fact better results
            #nn.InstanceNorm2d(int(self.output_dim/32/32))
            # https://arxiv.org/pdf/1812.04948.pdf

        )
        self.weights_init()
    def weights_init(m):
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
            torch.nn.init.normal_(m.weight, mean=0.0, std=0.02)
        if isinstance(m, nn.BatchNorm2d):
            torch.nn.init.normal_(m.weight, mean=0.0, std=0.02)
            torch.nn.init.constant_(m.bias, val=0)

    def forward(self, x,y):
        #print("0x",x.size())
        #print("0y",y.size())
        #x = torch.reshape(x,(self.batch_size,self.latent_dim,1,1))
        y = y.view([-1,self.num_of_classes,1,1])
        #print("1y",y.size())
        #x = torch.flatten(x, 1)
        #print("xflat",x.size()) # xflat torch.Size([64128])
        #x = torch.cat([x,y], 1)
        #print("1x",x.size()) ## 1x torch.Size([128, 501, 1, 1])
        # x = self.LeakyReLU(self.fc_1(x))
        #print("2x",x.size()) #2x torch.Size([128, 501])
        # x = self.LeakyReLU(self.fc_2(x))
        #x = torch.reshape(x,(self.hidden_dim,self.latent_dim,1,1))
        #print("3x",x.size())
        #x = x.view([-1,self.latent_dim+1,1,1])
        #print("xres",x.size())
        x = self.cntv_1(x)
        y = self.cntv_labels(y)
        #print("2x",x.size())
        #print("2y",y.size())
        x = torch.cat([x,y], 1)
        #print("3x",x.size())#
        x = self.cntv_2(x)
        # print("3",x.size())#
        x = self.cntv_3(x)
        # print("4",x.size())#
        x = self.cntv_4(x)
        #print("5",x.size())#
        x = self.cntv_5(x)
        #print("6",x.size())#
        x = self.cntv_6(x)
        #print("6",x.size())#
        x = x.view([-1, 3, 32, 32])
        return torch.tanh(x)

In [67]:
class Cond_Discriminator(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_of_classes,batch_size,SPARSITY_CNN,BOOST_STRENGTH,PERCENT_ON,relu_flag):
        super(Cond_Discriminator, self).__init__()
        self.input_dim = int(input_dim/32/32)
        self.hidden_dim = hidden_dim
        self.num_of_classes = num_of_classes
        self.relu_flag= relu_flag
        self.batch_size = batch_size
        self.SPARSITY_CNN = SPARSITY_CNN

        self.cnn_1 = nn.Sequential(
            nn.Conv2d(self.input_dim, self.hidden_dim, kernel_size=(3,3), stride=(2,2), padding=(1,1), bias=False),
            nn.LeakyReLU(drop_out, inplace=True)
        )

        self.cnn_labels = nn.Sequential(
            nn.Conv2d(self.num_of_classes, self.hidden_dim, kernel_size=(3,3), stride=(2,2), padding=(1,1), bias=False),
            nn.LeakyReLU(drop_out, inplace=True)
        )

        self.cnn_2 = nn.Sequential(
            nn.Conv2d(self.hidden_dim*2, self.hidden_dim*4, kernel_size=(4,4), stride=(2,2), padding=(1,1), bias=False),
        nn.BatchNorm2d(self.hidden_dim * 4),
        KWinners2d(channels=int(self.hidden_dim * 4), percent_on=PERCENT_ON, boost_strength=BOOST_STRENGTH,relu=self.relu_flag),
        #nn.LeakyReLU(0.2, inplace=True),
        nn.Dropout(drop_out),
        )

        self.cnn_3 = nn.Sequential(
            nn.Conv2d(self.hidden_dim*4, self.hidden_dim*8, kernel_size=(4,4), stride=(2,2), padding=(1,1), bias=False),
        nn.BatchNorm2d(self.hidden_dim * 8),
        KWinners2d(channels=int(self.hidden_dim * 8), percent_on=PERCENT_ON, boost_strength=BOOST_STRENGTH,relu=self.relu_flag),
        #nn.LeakyReLU(0.2, inplace=True),
        nn.Dropout(drop_out),
        )

        self.cnn_4 = nn.Sequential(
            nn.Conv2d(self.hidden_dim * 8, 1, kernel_size=(4,4), stride=(1,1), padding=(0,0), bias=False)
        )

        # self.sparse_cnn_4 = nn.Sequential(
        #     nn.Conv2d(self.hidden_dim * 4, self.input_dim + self.num_of_classes, kernel_size=(4,4), stride=(1,1), padding=(0,0), bias=False)
        # )

        # self.fc_1 = nn.Linear(2560, int(self.hidden_dim))
        # self.fc_2 = nn.Linear(int(self.hidden_dim), int(self.hidden_dim))
        # self.fc_out  = nn.Linear(int(self.hidden_dim),self.batch_size)
        #
        # self.LeakyReLU = nn.LeakyReLU(0.2)
    #     self.weights_init()
    def weights_init(m):
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
            torch.nn.init.normal_(m.weight, mean=0.0, std=0.02)
        if isinstance(m, nn.BatchNorm2d):
            torch.nn.init.normal_(m.weight, mean=0.0, std=0.02)
            torch.nn.init.constant_(m.bias, val=0)
    def forward(self, x, y):
        #print("y",y.size())
        #print("x",x.size())
        y = y.unsqueeze(2).repeat([1, 1, 32])
        y = y.unsqueeze(3).repeat([1, 1, 1, 32])
        #print("0",x.size())
        x = self.cnn_1(x)
        y = self.cnn_labels(y)
        x = torch.cat([x,y], 1)
        #print("1",x.size())
        x = self.cnn_2(x)
        x = self.cnn_3(x)
        #print("2",x.size())
        x = self.cnn_4(x)
        #print("3",x.size())# 3 torch.Size([512, 4, 1, 1])
        #print("y",y.size())# y torch.Size([512, 1])
        #y = torch.reshape(y,(self.batch_size,1,1,1))
        #print("yr",y.size())
        # x = torch.reshape(x,(512,4))
        #
        # #print("xf",x.size())
        # x = torch.cat([x,y], 1)
        # x = torch.flatten(x)
        #
        #
        # #print("4",x.size()) # 4 torch.Size([2560])
        # x = self.LeakyReLU(self.fc_1(x))
        # #print("5",x.size())
        # x = self.LeakyReLU(self.fc_2(x))
        # #print("6",x.size())
        # x = self.fc_out(x)
        #print("4",x.size())
        return torch.sigmoid(x)

In [68]:

cond_generator = Cond_Generator(batch_size=batch_size,latent_dim=latent_dim, hidden_dim=128, output_dim=img_size, num_of_classes=len(dataset.classes),SPARSITY_CNN=SPARSITY_CNN,BOOST_STRENGTH=BOOST_STRENGTH,PERCENT_ON=PERCENT_ON,relu_flag=relu_flag).to(device)
cond_discriminator = Cond_Discriminator( hidden_dim=64, input_dim=img_size, num_of_classes=len(dataset.classes),batch_size=batch_size,SPARSITY_CNN=SPARSITY_CNN,BOOST_STRENGTH=BOOST_STRENGTH,PERCENT_ON=PERCENT_ON,relu_flag=relu_flag).to(device)

# Optimizers
generator_optimizer = torch.optim.Adam(cond_generator.parameters(), lr=0.0001)
# generator_optimizer = torch.optim.RMSprop(cond_generator.parameters(), lr=0.00005)
generator_scheduler = optim.lr_scheduler.ExponentialLR(optimizer=generator_optimizer, gamma=0.99)
discriminator_optimizer = torch.optim.Adam(cond_discriminator.parameters(), lr=0.0001)
# discriminator_optimizer = torch.optim.RMSprop(cond_discriminator.parameters(), lr=0.00005)
discriminator_scheduler = optim.lr_scheduler.ExponentialLR(optimizer=discriminator_optimizer, gamma=0.99)

# loss
#criterion = nn.MSELoss()
#criterion  = nn.BCELoss(reduction="mean")
#criterion = nn.BCEWithLogitsLoss()

In [69]:
fixed_noise = torch.randn(32, latent_dim,1,1,device=device)
fixed_labels = torch.randint(len(dataset.classes),(32,),device=device)
fixed_labels = F.one_hot(fixed_labels, len(dataset.classes)).float()

In [70]:
from torch import autograd
# https://arxiv.org/pdf/1704.00028.pdf
def compute_gp(netD, real_data, fake_data,rand_y):
        batch_size = real_data.size(0)
        eps = torch.rand(batch_size, 1, 1, 1).to(real_data.device)
        eps = eps.expand_as(real_data)
        interpolation = eps * real_data + (1 - eps) * fake_data
        interp_logits = netD(interpolation,rand_y)
        grad_outputs = torch.ones_like(interp_logits)
        gradients = autograd.grad(
            outputs=interp_logits,
            inputs=interpolation,
            grad_outputs=grad_outputs,
            create_graph=True,
            retain_graph=True,
        )[0]

        # Compute and return Gradient Norm
        gradients = gradients.view(batch_size, -1)
        grad_norm = gradients.norm(2, 1)
        return torch.mean((grad_norm - 1) ** 2)

## Trening

In [83]:
G_losses = []
D_losses = []

GPATH = "WGAN-Clipping-K-Win_G0.pt"
DPATH = "WGAN-Clipping-K-Win_D0.pt"


################# NO. PARAMS ####################
G_total_params = sum(p.numel() for p in cond_generator.parameters() if p.requires_grad)
print("No. of Parametres - G : ", G_total_params)
G_total_params = sum(p.numel() for p in cond_generator.parameters() if p.requires_grad)
print("No. of Trainable parametres - G : ", G_total_params)

D_total_params = sum(p.numel() for p in cond_discriminator.parameters() if p.requires_grad)
print("No. of Parametres - D : ", D_total_params)
D_total_params = sum(p.numel() for p in cond_discriminator.parameters() if p.requires_grad)
print("No. of Trainable parametres - D : ", D_total_params)
################# NO. PARAMS ####################

num_epochs = 61
for epoch in range(num_epochs):
    # For each batch in the dataloader
    discriminator_fake_acc = []
    discriminator_real_acc = []
    for i, data in enumerate(train_loader, 0):

        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        discriminator_optimizer.zero_grad()
        # Format batch
        real_images = data[0].to(device)
        b_size = real_images.size(0)
        for _ in range(disc_iter):
            y = data[1]
            y = F.one_hot(y, num_classes=len(dataset.classes)).to(device).float()

            #####label = torch.full((b_size,),0.9, dtype=torch.float, device=device) # Setting non 1 labels for real images & preventing overconfidence
            # Forward pass real batch through D
            real_preds = cond_discriminator(real_images, y).view(-1)
            # Calculate loss on all-real batch
            #error_discriminator_real = -criterion(real_preds, label)
            error_discriminator_real = -torch.mean(real_preds)
            # Calculate gradients for D in backward pass
            #error_discriminator_real.backward()
            discriminator_real_acc.append(real_preds.mean().item())

            ## Train with all-fake batch
            # Generate batch of latent vectors
            noise = torch.randn(b_size, latent_dim,1,1,device=device)
            rand_y = torch.randint(len(dataset.classes),(b_size,),device=device)
            rand_y = F.one_hot(rand_y, len(dataset.classes)).float()
            # Generate fake image batch with Generator
            fake_images = cond_generator(noise, rand_y)
            #####label_fake = torch.full((b_size,),0.05, dtype=torch.float, device=device)# Setting non 0 labels for fake images & preventing overconfidence
            # Classify all fake batch with Discriminator
            fake_preds = cond_discriminator(fake_images.detach(), rand_y).view(-1)
            # Calculate D's loss on the all-fake batch
            #error_discriminator_fake = criterion(fake_preds, label_fake)
            error_discriminator_fake = torch.mean(fake_preds)
            # Calculate the gradients for this batch, accumulated (summed) with previous gradients
            #error_discriminator_fake.backward()
            discriminator_fake_acc.append(fake_preds.mean().item())
            # Compute gradient Penalty
            #gp = compute_gp(cond_discriminator,real_images,fake_images,rand_y)
            # Compute error of D as sum over the fake and the real batches
            error_discriminator = error_discriminator_real + error_discriminator_fake #+ gp*0.2
            # Update D
            error_discriminator.backward()
            discriminator_optimizer.step()
            D_losses.append(error_discriminator.item())
            # Weigth clipping from WGAN Paper (with code)
            # for p in cond_discriminator.parameters():
            #     p.data.clamp_(-WEIGHT_CLIP , WEIGHT_CLIP )
        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        generator_optimizer.zero_grad()
        noise = torch.randn(b_size, latent_dim,1,1,device=device)
        rand_y = torch.randint(len(dataset.classes),(b_size,),device=device)
        rand_y = F.one_hot(rand_y, len(dataset.classes)).float()
        fake_images = cond_generator(noise, rand_y)

        label = torch.full((b_size,),1., dtype=torch.float, device=device)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        predictions = cond_discriminator(fake_images, rand_y).view(-1)
        # Calculate G's loss based on this output
        #error_generator = -criterion(predictions, label)
        error_generator = -torch.mean(predictions)
        # Calculate gradients for G
        error_generator.backward()

        D_G_z2 = predictions.mean().item()
        # Update G
        generator_optimizer.step()
        # Output training stats
        # Save Losses for plotting later
        G_losses.append(error_generator.item())



    print(f"Epoch: {epoch}, discrimiantor fake error: {np.mean(discriminator_fake_acc):.3}, discriminator real acc: {np.mean(discriminator_real_acc):.3}")
    generator_scheduler.step()
    discriminator_scheduler.step()
    cond_generator.apply(update_boost_strength)
    cond_discriminator.apply(update_boost_strength)
    if epoch % 10 == 0:
        with torch.no_grad():
            fake = cond_generator(fixed_noise, fixed_labels).detach().cpu()
        grid = torchvision.utils.make_grid(fake,nrow=8, normalize=True)
        grid = grid.permute(1, 2, 0)
        plt.figure(figsize=(10,10))
        plt.title(f"Generations")
        plt.imshow(grid)
        plt.axis('off')
        plt.show()
        # SAVE MODEL
        torch.save(cond_generator, GPATH)
        torch.save(cond_discriminator, DPATH)

No. of Parametres - G :  7198086
No. of Trainable parametres - G :  7198086
No. of Parametres - D :  2633472
No. of Trainable parametres - D :  2633472


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

## Generowanie wyników

In [75]:
# LOAD MODELS
# cond_generator = torch.load(GPATH)
# cond_discriminator = torch.load(DPATH)
# cond_generator.eval()
# cond_discriminator.eval()

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [84]:
def generate_images(number_of_images, clazz=None, show=True):
    fixed_noise = torch.randn(number_of_images, latent_dim, 1, 1, device=device)
    fixed_labels = torch.randint(len(dataset.classes), (number_of_images,), device=device)
    if clazz:
        fixed_labels = torch.tensor([clazz for _ in range(number_of_images)], device=device)
    fixed_labels = F.one_hot(fixed_labels, len(dataset.classes)).float()
    with torch.no_grad():
        fake = cond_generator(fixed_noise, fixed_labels).detach().cpu()
        # normalizacja z [-1, 1] do [0, 1]
        fake = (fake + 1) / 2

        if show:
            grid = torchvision.utils.make_grid(fake)
            grid = grid.permute(1, 2, 0)
            plt.figure(figsize=(10,10))
            plt.title(f"Generations")
            plt.imshow(grid)
            plt.axis('off')
            plt.show()

        return fake

In [85]:
images = generate_images(1000, show=False)

RuntimeError: philox_cuda_state for an unexpected CUDA generator used during capture. In regions captured by CUDA graphs, you may only use the default CUDA RNG generator on the device that's current when capture begins. If you need a non-default (user-supplied) generator, or a generator on another device, please file an issue.